In [ ]:
# Install the latest JAXlib version.
# This code was originally tested on the 0.4.3 version, then updated to the 0.4.12
!pip install --upgrade -q pip jax jaxlib

## ⚖️ Choose A or C:

## A: Emulating multi-device system on CPU

Use this section to initialize a set of virtual devices on CPU if you have no access to a multi-device system.

It can also help you prototype, debug and test your multi-device code locally before running it on the expensive system.

Even in the case of using Google Colab it can help you prototype faster because a CPU runtime is faster to restart.

In [ ]:
import os
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8'

In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
jax.devices()

[CpuDevice(id=0),
 CpuDevice(id=1),
 CpuDevice(id=2),
 CpuDevice(id=3),
 CpuDevice(id=4),
 CpuDevice(id=5),
 CpuDevice(id=6),
 CpuDevice(id=7)]

## (do not use) B: Setting up Colab TPU

#### [this section creates Colab TPU, which at the moment does not work with pjit(), we need the Cloud TPU (section C) instead of Colab TPU]

Use this section if you want to use Google Cloud TPU (and don't forget to change the Runtime type in "Runtime"-> "Change runtime type" -> "TPU".

In [ ]:
# in order to use TPU you have to run this cell before importing JAX
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [ ]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

tpu


In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

## C: Using Cloud TPU and a Local runtime in Colab

Make preparations according to the following links:

* Creating a Cloud TPU https://cloud.google.com/tpu/docs/managing-tpus-tpu-vm#tpu-vms

* Preparing Jupyter and connect to a Local runtime https://research.google.com/colaboratory/local-runtimes.html


In [ ]:
!pip install 'jax[tpu]>=0.2.16' -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/libtpu_releases.html


In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

## Using pjit()

In [ ]:
jax.__version__

'0.4.3'

In [ ]:
from jax.experimental.pjit import pjit

In [ ]:
from jax import random

### Old vmap+pmap example

In [ ]:
def dot(v1, v2):
  return jnp.vdot(v1, v2)

In [ ]:
rng_key = random.PRNGKey(42)

In [ ]:
vs = random.normal(rng_key, shape=(20_000_000,3))
v1s = vs[:10_000_000,:]
v2s = vs[10_000_000:,:]

v1s.shape, v2s.shape

((10000000, 3), (10000000, 3))

In [ ]:
v1sp = v1s.reshape((8, v1s.shape[0]//8, v1s.shape[1]))
v2sp = v2s.reshape((8, v2s.shape[0]//8, v2s.shape[1]))

v1sp.shape, v2sp.shape

((8, 1250000, 3), (8, 1250000, 3))

In [ ]:
dot_parallel = jax.pmap(
    jax.vmap(dot, in_axes=(0,0)),
    in_axes=(0,0)
)

In [ ]:
x_pmap = dot_parallel(v1sp,v2sp)

In [ ]:
x_pmap.shape

(8, 1250000)

In [ ]:
x_pmap = x_pmap.reshape((x_pmap.shape[0]*x_pmap.shape[1]))
x_pmap.shape

(10000000,)

### Replacing with pjit and 1D mesh

In [ ]:
from jax.sharding import Mesh
from jax.sharding import PartitionSpec

In [ ]:
import numpy as np

In [ ]:
devices = np.array(jax.devices())
devices

array([TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
       TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
       TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
       TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
       TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
       TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
       TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
       TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)],
      dtype=object)

Non-vectorized function won't work here because output partitioning works only for rank>=1 tensors

In [ ]:
f = pjit(dot,
         in_axis_resources=None,
         out_axis_resources=PartitionSpec('devices')
         )

In [ ]:
with Mesh(devices, ('devices',)):
  x_pjit=f(v1s,v2s)

2023-02-11 20:27:04.753898: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:476] cache miss fail: ValueError: One of pjit outputs is incompatible with its sharding annotation NamedSharding(mesh={'devices': 8}, spec=PartitionSpec('devices',)): Sharding NamedSharding(mesh={'devices': 8}, spec=PartitionSpec('devices',)) is only valid for values of rank at least 1, but was applied to a value of rank 0

At:
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(866): pjit_check_aval_sharding
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(839): _pjit_jaxpr
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/linear_util.py(301): memoized_fun
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(411): common_infer_params
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(610): infer_params
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(115): _python_pjit_helper
  /home/grigo/.local/lib/python3.8/

ValueError: ignored

Using a vectorized function that produces rank 1 tensor output. Input is replicated across all the devices.

In [ ]:
f = pjit(jax.vmap(dot),
         in_axis_resources=None,
         out_axis_resources=PartitionSpec('devices')
         )

In [ ]:
with Mesh(devices, ('devices',)):
  x_pjit=f(v1s,v2s)

In [ ]:
x_pjit.shape

(10000000,)

In [ ]:
jax.numpy.all(x_pjit == x_pmap)

Array(True, dtype=bool)

Input is sharded across all the devices.

In [ ]:
f = pjit(jax.vmap(dot),
         in_axis_resources=PartitionSpec('devices'),
         out_axis_resources=PartitionSpec('devices')
         )

In [ ]:
with Mesh(devices, ('devices',)):
  x_pjit=f(v1s,v2s)

In [ ]:
x_pjit.shape

(10000000,)

In [ ]:
jax.numpy.all(x_pjit == x_pmap)

Array(True, dtype=bool)

The same as previous

In [ ]:
f = pjit(jax.vmap(dot),
         in_axis_resources=(PartitionSpec('devices'), None),
         out_axis_resources=PartitionSpec('devices')
         )

In [ ]:
with Mesh(devices, ('devices',)):
  x_pjit=f(v1s,v2s)

In [ ]:
x_pjit.shape

(10000000,)

Also the same

In [ ]:
f = pjit(jax.vmap(dot),
         in_axis_resources=PartitionSpec('devices', None),
         out_axis_resources=PartitionSpec('devices')
         )

In [ ]:
with Mesh(devices, ('devices',)):
  x_pjit=f(v1s,v2s)

In [ ]:
x_pjit.shape

(10000000,)

Trying more values in PartitionSpec than there are parameters in the function

In [ ]:
f = pjit(jax.vmap(dot),
         in_axis_resources=(PartitionSpec('devices'), None, None),
         out_axis_resources=PartitionSpec('devices')
         )

In [ ]:
with Mesh(devices, ('devices',)):
  x_pjit=f(v1s,v2s)

2023-02-11 20:27:37.482894: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:476] cache miss fail: ValueError: pjit in_axis_resources specification must be a tree prefix of the positional arguments tuple passed to the `pjit`-decorated function. In particular, pjit in_axis_resources must either be a None, a PartitionSpec, or a tuple of length equal to the number of positional arguments. But pjit in_axis_resources is the wrong length: got a tuple or list of length 3 for an args tuple of length 2.

At:
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(706): flatten_axis_resources
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(739): _process_in_axis_resources
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(407): common_infer_params
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(610): infer_params
  /home/grigo/.local/lib/python3.8/site-packages/jax/_src/pjit.py(115): _python_pjit_helper
  /home/grigo/.l

ValueError: ignored

Finally sharding both input parameters

In [ ]:
f = pjit(jax.vmap(dot),
         in_axis_resources=(PartitionSpec('devices'), PartitionSpec('devices')),
         out_axis_resources=PartitionSpec('devices')
         )

In [ ]:
with Mesh(devices, ('devices',)):
  x_pjit=f(v1s,v2s)

In [ ]:
x_pjit.shape

(10000000,)

### 2D mesh case

In [ ]:
from jax.sharding import PartitionSpec as P   # could be useful to reduce typing
from jax.sharding import Mesh
import numpy as np

In [ ]:
rng_key = random.PRNGKey(42)

In [ ]:
vs = random.normal(rng_key, shape=(8_000,10_000))
v1s = vs[:4_000,:]
v2s = vs[4_000:,:]

v1s.shape, v2s.shape

((5000, 10000), (5000, 10000))

In [ ]:
devices = np.array(jax.devices()).reshape(2, 4)
devices

array([[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
        TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
        TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
        TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1)],
       [TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
        TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
        TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
        TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]],
      dtype=object)

In [ ]:
def dot(v1, v2):
  return jnp.vdot(v1, v2)

In [ ]:
f = pjit(jax.vmap(dot),
         in_axis_resources=(P('x', 'y'), P('x', 'y')),
         out_axis_resources=P('x')
         )

In [ ]:
with Mesh(devices, ('x','y')):
  x_pjit=f(v1s,v2s)

In [ ]:
x_pjit.shape

(5000,)

### Looking into HLO

You can't see any collective ops on the jaxpr level:

In [ ]:
with Mesh(devices, ('x','y')):
  print(jax.make_jaxpr(f)(v1s,v2s))

{ lambda ; a:f32[5000,10000] b:f32[5000,10000]. let
    c:f32[5000] = pjit[
      donated_invars=(False, False)
      in_positional_semantics=(<_PositionalSemantics.GLOBAL: 1>, <_PositionalSemantics.GLOBAL: 1>)
      in_shardings=(OpShardingSharding({devices=[2,4]0,1,2,3,4,5,6,7}), OpShardingSharding({devices=[2,4]0,1,2,3,4,5,6,7}))
      inline=False
      jaxpr={ lambda ; d:f32[5000,10000] e:f32[5000,10000]. let
          f:f32[5000] = dot_general[dimension_numbers=(([1], [1]), ([0], [0]))] d
            e
        in (f,) }
      keep_unused=False
      name=dot
      out_positional_semantics=_PositionalSemantics.GLOBAL
      out_shardings=(OpShardingSharding({devices=[2,4]0,1,2,3,4,5,6,7 last_tile_dim_replicate}),)
      resource_env=ResourceEnv(Mesh(device_ids=array([[0, 1, 2, 3],
       [4, 5, 6, 7]]), axis_names=('x', 'y')), ())
    ] a b
  in (c,) }


But you can see them after the compilation:

In [ ]:
# Thanks to https://github.com/google/jax/discussions/11275
with Mesh(devices, ('x','y')):
    modules = f.lower(v1s, v2s).compile().compiler_ir()
    for hlo in modules:
        print(hlo.to_string())

HloModule pjit_dot, is_scheduled=true, entry_computation_layout={(f32[2500,2500]{1,0:T(8,128)},f32[2500,2500]{1,0:T(8,128)})->f32[2500]{0:T(1024)}}

%scalar_add_computation (scalar_lhs: f32[], scalar_rhs: f32[]) -> f32[] {
  %scalar_rhs = f32[]{:T(256)} parameter(1)
  %scalar_lhs = f32[]{:T(256)} parameter(0)
  ROOT %add = f32[]{:T(256)} add(f32[]{:T(256)} %scalar_lhs, f32[]{:T(256)} %scalar_rhs)
}

%fused_computation (param_0.2: f32[2500,2500], param_1.2: f32[2500,2500]) -> f32[2500] {
  %param_0.2 = f32[2500,2500]{1,0:T(8,128)} parameter(0)
  %param_1.2 = f32[2500,2500]{1,0:T(8,128)} parameter(1)
  %multiply.2 = f32[2500,2500]{1,0:T(8,128)} multiply(f32[2500,2500]{1,0:T(8,128)} %param_0.2, f32[2500,2500]{1,0:T(8,128)} %param_1.2)
  %constant.2 = f32[]{:T(256)} constant(0)
  ROOT %reduce.2 = f32[2500]{0:T(1024)} reduce(f32[2500,2500]{1,0:T(8,128)} %multiply.2, f32[]{:T(256)} %constant.2), dimensions={1}, to_apply=%scalar_add_computation, metadata={op_name="pjit(dot)/jit(main)/dot_gene

## MLP example

### Preparing data

Install these modules if you created a new empty cloud machine

In [ ]:
!pip install tensorflow

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |█████████████████▍              | 319.5 MB 144.1 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▎| 574.6 MB 276 kB/s eta 0:00:50

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 588.3 MB 8.9 kB/s 
     |████████████████████████████████| 126 kB 102.5 MB/s 
     |████████████████████████████████| 57 kB 8.9 MB/s 
     |████████████████████████████████| 4.8 MB 91.7 MB/s 
     |████████████████████████████████| 4.7 MB 90.7 MB/s 
     |████████████████████████████████| 1.7 MB 96.7 MB/s 
     |████████████████████████████████| 21.5 MB 94.9 MB/s 
     |████████████████████████████████| 1.1 MB 90.7 MB/s 
     |████████████████████████████████| 6.0 MB 82.4 MB/s 
     |████████████████████████████████| 439 kB 102.2 MB/s 
     |████████████████████████████████| 81 kB 13.8 MB/s 
     |████████████████████████████████| 2.4 MB 95.6 MB/s 
     |████████████████████████████████| 177 kB 91.4 MB/s 
     |████████████████████████████████| 93 kB 2.6 MB/s 
     |████████████████████████████████| 4.9 MB 95.7 MB/s 
     |████████████████████████████████| 781 kB 94.4 MB/s 
     |████████████████████████████████| 232 kB 101.2 MB/s 


In [ ]:
!pip install tensorflow_datasets

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |████████████████████████████████| 5.3 MB 4.9 MB/s 
     |████████████████████████████████| 110 kB 104.8 MB/s 
     |████████████████████████████████| 152 kB 110.1 MB/s 
     |████████████████████████████████| 146 kB 107.6 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 78 kB 11.4 MB/s 
     |████████████████████████████████| 223 kB 103.7 MB/s 
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21485 sha256=73c0616d47fbaafbf82f3d1905e5301756920f772bc9c73de9dc34ab49496b93
  Stored in directory: /home/grigo/.cache/pi

In [ ]:
import jax
import tensorflow as tf
import tensorflow_datasets as tfds

data_dir = '/tmp/tfds'

data, info = tfds.load(name="mnist",
                       data_dir=data_dir,
                       as_supervised=True,
                       with_info=True)

data_train = data['train']
data_test  = data['test']

/home/grigo/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.19 file/s]
2023-02-11 20:30:05.457321: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-02-11 20:30:05.457357: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)


Dataset mnist downloaded and prepared to /tmp/tfds/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
HEIGHT = 28
WIDTH  = 28
CHANNELS = 1
NUM_PIXELS = HEIGHT * WIDTH * CHANNELS
NUM_LABELS = info.features['label'].num_classes
NUM_DEVICES = jax.device_count()
BATCH_SIZE  = 32

In [ ]:
def preprocess(img, label):
  """Resize and preprocess images."""
  return (tf.cast(img, tf.float32)/255.0), label

train_data = tfds.as_numpy(
    data_train.map(preprocess).batch(NUM_DEVICES*BATCH_SIZE).prefetch(1)
)
test_data  = tfds.as_numpy(
    data_test.map(preprocess).batch(NUM_DEVICES*BATCH_SIZE).prefetch(1)
)

In [ ]:
len(train_data)

235

### Preparing MLP

In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap, value_and_grad
from jax import random
from jax.nn import swish, logsumexp, one_hot

In [ ]:
LAYER_SIZES = [28*28, 512, 10]
PARAM_SCALE = 0.01

In [ ]:
def init_network_params(sizes, key=random.PRNGKey(0), scale=1e-2):
  """Initialize all layers for a fully-connected neural network with given sizes"""

  def random_layer_params(m, n, key, scale=1e-2):
    """A helper function to randomly initialize weights and biases of a dense layer"""
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k, scale) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

init_params = init_network_params(LAYER_SIZES, random.PRNGKey(0), scale=PARAM_SCALE)

In [ ]:
def predict(params, image):
  """Function for per-example predictions."""
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = swish(outputs)

  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b
  return logits

batched_predict = vmap(predict, in_axes=(None, 0))

### Loss and update functions

In [ ]:
from jax.experimental.pjit import pjit
from jax.sharding import PartitionSpec as P
from jax.sharding import Mesh
import numpy as np

In [ ]:
INIT_LR = 1.0
DECAY_RATE = 0.95
DECAY_STEPS = 5
NUM_EPOCHS  = 20

In [ ]:
def loss(params, images, targets):
  """Categorical cross entropy loss function."""
  logits = batched_predict(params, images)
  log_preds = logits - logsumexp(logits) # logsumexp trick https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
  return -jnp.mean(targets*log_preds)

def update(params, x, y, epoch_number):
  loss_value, grads = value_and_grad(loss)(params, x, y)
  lr = INIT_LR * DECAY_RATE ** (epoch_number / DECAY_STEPS)
  return [(w - lr * dw, b - lr * db)
          for (w, b), (dw, db) in zip(params, grads)], loss_value

In [ ]:
f_update = pjit(update,
         in_axis_resources=(None, P('x'), P('x'), None),
         out_axis_resources=None
         )

### Training loop

In [ ]:
devices = np.array(jax.devices())
devices

array([TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
       TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
       TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
       TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
       TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
       TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
       TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
       TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)],
      dtype=object)

In [ ]:
def batch_accuracy(params, images, targets):
  images = jnp.reshape(images, (len(images), NUM_PIXELS))
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == targets)

f_batch_accuracy = pjit(batch_accuracy,
         in_axis_resources=(None, P('x'), P('x')),
         out_axis_resources=None
         )

def accuracy(params, data):
  accs = []
  for images, targets in data:
    accs.append(f_batch_accuracy(params, images, targets))
  return jnp.mean(jnp.array(accs))

! Be careful. This example does not work with Colab TPU because of Colab bug, it only works with Cloud TPU.

https://github.com/google/jax/issues/8300

If you want to use TPU, you need to build your own runtime on Google Cloud Platform (https://cloud.google.com/tpu/docs/jax-pods) and connects to it using Jupyter (https://research.google.com/colaboratory/local-runtimes.html).

In [ ]:
import time

params = init_params
with Mesh(devices, ('x',)):
  for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    losses = []
    for x, y in train_data:
      x = jnp.reshape(x, (len(x), NUM_PIXELS))
      y = one_hot(y, NUM_LABELS)
      params, loss_value = f_update(params, x, y, epoch)
      losses.append(jnp.sum(loss_value))
    epoch_time = time.time() - start_time

    train_acc = accuracy(params, train_data)
    test_acc = accuracy(params, test_data)
    print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
    print("Training set loss {}".format(jnp.mean(jnp.array(losses))))
    print("Training set accuracy {}".format(train_acc))
    print("Test set accuracy {}".format(test_acc))

Epoch 0 in 1.83 sec
Training set loss 0.6914460062980652
Training set accuracy 0.8648548126220703
Test set accuracy 0.874804675579071
Epoch 1 in 0.98 sec
Training set loss 0.6259850859642029
Training set accuracy 0.8882258534431458
Test set accuracy 0.893847644329071
Epoch 2 in 0.99 sec
Training set loss 0.6164931058883667
Training set accuracy 0.8978224396705627
Test set accuracy 0.901074230670929
Epoch 3 in 0.96 sec
Training set loss 0.6100647449493408
Training set accuracy 0.906527042388916
Test set accuracy 0.908984363079071
Epoch 4 in 1.05 sec
Training set loss 0.6044379472732544
Training set accuracy 0.9142231941223145
Test set accuracy 0.917285144329071
Epoch 5 in 1.06 sec
Training set loss 0.5995732545852661
Training set accuracy 0.9211103320121765
Test set accuracy 0.9248046875
Epoch 6 in 0.95 sec
Training set loss 0.5955986976623535
Training set accuracy 0.9261746406555176
Test set accuracy 0.929394543170929
Epoch 7 in 0.95 sec
Training set loss 0.5924407839775085
Training se

Some resources that might help you achieve weight sharding as well:
https://github.com/google/jax/discussions/8649

**!!! Do not forget to shutdown your Cloud TPU, or you'll spend much money on it!!!**